## Title

The purpose of this notebook is to compare the errors associated with fuel moisture models when using interpolated data from RTMA versus actual data from RAWS sensors. The main focus is on precipitation. The other variables used from the RTMA data are relative humidity (RH) and temperature. These are expected to be smooth across space, while precipitation can vary widely even across small spatial scales.

### Setup

<mark>**Question for Jan:** best way to have meso token in code?<\mark>

In [ ]:
# Environment
import numpy as np
import pandas as pd
from MesoPy import Meso
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, time
import pytz
# Local modules for handling data and running moisture models
import data_funcs as datf
import moisture_models as mod

meso_token="4192c18707b848299783d59a9317c6e1"
m=Meso(meso_token)

### Data Read

Below we print all the RAWS stations in Colorado with full data availability over the time period in question. The search and output is done in the notebook `find_RAWS_station`.,

Note: the time period is arbitrary at this point, and the `find_RAWS_station` notebook summarized this data availability for this particular time period. We need to consider how this should work for a different time period. Question: will a particular STID give different available data series for different times, or always the same data series names?

<mark>**Question for Jan:** should we include the RAWS station search code here? For the following I'm just using an arbitrary station CPTC2<\mark>

In [ ]:
vars='air_temp,relative_humidity,precip_accum,fuel_moisture'
time_start = "201806010800"  # June 1 2018 08:00 in format yyyymmddHHMM
time_end   = "201907200900"  # June 20 2018 09:00 in format yyyymmddHHMM 

meso_ts = m.timeseries(time_start, time_end, 
                       stid="CPTC2", vars=vars)
station = meso_ts['STATION'][0]

Organize RAWS data.

In [ ]:
import data_funcs as datf

raws_dat = datf.format_raws(station)

In [ ]:
%matplotlib inline
plt.figure(figsize=(16,4))
plt.plot(raws_dat['rh'],linestyle='-',c='k')
plt.title(station['STID'] + ' rh')
plt.xlabel('Time (hours)') 
plt.ylabel('Relative Humidity (%)')

In [ ]:
%matplotlib inline
plt.figure(figsize=(16,4))
plt.plot(raws_dat['fm'],linestyle='-',c='k',label='10-h fuel data')
plt.title(station['STID'] + ' 10 h fuel moisture data')
plt.xlabel('Time (hours)') 
plt.ylabel('Fuel moisture content (%)')
plt.legend()

## Run Augmented Moisture Model with RAWS Data

In [ ]:
## Model params

hours=1200 # total simulation
h2 = 300
m = np.zeros(hours) # preallocate
m[0]= raws_dat['fm'][0]             # initial state  
P = np.zeros(hours)
P[0] = 1e-3 # background state variance
H = np.array([1.])   # all oQ = np.array([0.02]) # process noise variancebserved
Q = np.array([1e-3]) # process noise variance
R = np.array([1e-3]) # data variance

Augmented Model

In [ ]:
m,Ec = mod.run_augmented_kf(raws_dat,h2,hours)  # extract from state

In [ ]:
def plot_moisture(hmin,hmax):
  print('training from 0 to',h2,'plot from',hmin,'to',hmax)
  plt.figure(figsize=(16,4))
  plt.plot(range(hmin,hmax),raws_dat['Ed'][hmin:hmax],linestyle='--',c='r',label='Drying Equilibrium (%)')
  plt.plot(range(hmin,hmax),raws_dat['Ew'][hmin:hmax],linestyle='--',c='b',label='Wetting Equilibrium (%)')
  plt.plot(range(hmin,hmax),Ec[hmin:hmax],linestyle='--',c='g',label='Equilibrium Correction (%)')
  plt.plot(range(hmin,hmax),m[hmin:hmax],linestyle='-',c='k',label='filtered')
  plt.plot(range(hmin,hmax),raws_dat['fm'][hmin:hmax],linestyle='-',c='b',label='RAWS data (%)')
  plt.plot(range(hmin,hmax),raws_dat['rain'][hmin:hmax],linestyle='-',c='b',label='RTMA rain (mm/h)')
  if hmin>=h2:
    plt.plot(m[hmin:h2],linestyle='-',c='k',label='Filtered')
  h1 = np.maximum(hmin,h2)
  plt.plot(range(h1,hmax),m[h1:hmax],linestyle='-',c='r',label='Forecast (%)')
  plt.title(station['STID'] +' Kalman filtering and forecast with augmented state, real data. Training 0:%i hmax' % h2)
  plt.xlabel('Time (hours)') 
  plt.ylabel('Fuel moisture content (%)')
  plt.legend()

In [ ]:
plot_moisture(0,hours)

In [ ]:
plot_moisture(1000,hours)

## Model Validation

Calculate Mean Absolute Prediction Error (MAPE) for the forecast versus observed fuel moisture data. For comparison, I will calculate MAPE for the entire series, the forecast period, and just the final value (where errors may have accumulated up to).

In [ ]:
def mape(a, b):
    return ((a - b).__abs__()).mean()

In [ ]:
print('Total MAPE: '+ str(np.round(mape(raws_dat['fm'][0:hours], m), 5)))
print('-'*25)
print('Train Period: '+ str(np.round(mape(raws_dat['fm'][0:300], m[0:300]), 5)))
print('-'*25)
print('Test Period: '+ str(np.round(mape(raws_dat['fm'][301:hours], m[301:hours]), 5)))
print('-'*25)
print('Final Time: '+ str(np.round(mape(raws_dat['fm'][-1], m[-1]), 5)))

In [ ]:
def mse(a, b):
    return ((a - b)**2).mean()

In [ ]:
print('Total MAPE: '+ str(np.round(mse(raws_dat['fm'][0:hours], m), 5)))
print('-'*25)
print('Train Period: '+ str(np.round(mse(raws_dat['fm'][0:300], m[0:300]), 5)))
print('-'*25)
print('Test Period: '+ str(np.round(mse(raws_dat['fm'][301:hours], m[301:hours]), 5)))
print('-'*25)
print('Final Time: '+ str(np.round(mse(raws_dat['fm'][-1], m[-1]), 5)))